In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

In [3]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [36]:
def geo(word):
    word = word['keyword'].lower() # если вдруг назваание написали с заглавной буквы
    for i in list(geo_data): # цикл который перебирает все варианты и если найдено совпадение возвращает регион
        if word in geo_data[i]:
            return i
        else:
            pass
    return 'undefined'      

In [38]:
stats['geo']= stats.apply(geo, axis=1)
stats.geo.value_counts()

undefined         99991
Северо-Запад          3
Центр                 3
Дальний Восток        3
Name: geo, dtype: int64

In [21]:
'москва' in geo_data['Центр']

True

**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [55]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
def rating_class(row):
    if int(row['rating'])<=2:
        return 'низкий рейтинг'
    elif int(row['rating'])<=4:
        return 'средний рейтинг'
    elif int(row['rating'])>=4.5:
        return 'высокий рейтинг'
    else:
        return '-'
ratings['rating_name'] = ratings.apply(rating_class, axis=1)
ratings.rating_name.value_counts()

средний рейтинг    67075
низкий рейтинг     17834
высокий рейтинг    15095
Name: rating_name, dtype: int64

**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [94]:
data = pd.read_csv('ml-latest-small/ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [118]:
group_id = data.groupby('userId').count()
target_id = group_id[group_id['movieId']>=100].reset_index().userId.tolist()
data = data[data.userId.isin(target_id)]

In [136]:
data_p = data.groupby('userId').max().merge(data.groupby('userId').min(), on= 'userId', how = 'left').reset_index()
data_p['dif'] = data_p.apply(lambda x: x['timestamp_x']-x['timestamp_y'], axis=1)

In [142]:
data_p.dif.mean()

39331748.56653992